In [10]:
import pandas as pd
import networkx as nx
from helpers import get_refs_and_cites, deduplicate_edges
from tqdm import tqdm
from IPython.display import clear_output
from habanero import Crossref
import pickle

Теперь давайте сделаем граф. На этом этапе я решил не делить данные по годам, поскольку пока мы не изучаем динамику и данных в принципе немного.

Для этого я воспользуюсь библиотекой networkx. 

In [2]:
cr = Crossref()
def recursive_edges(base_doi: str, edges: list, doi: str = "", depth = 3, curr_depth = 3):
    """
    Build the citation graph, starting from doi, with depth.
    Need to deduplicate edges after.
    Hoping that fixing the depth to 5 would yield a single connected component for multiple starting dois
    """
    if curr_depth == 0:
        return
    # time.sleep(0.5)
    d = doi if len(doi) else base_doi
    try:
        refs, cites = get_refs_and_cites(d, cr)
    except Exception as e:
        print(e)
        print(type(e))
    refs_edges = zip([d] * len(refs), refs)
    cites_edges = zip(cites, [d] * len(cites))
    edges.extend(refs_edges)
    edges.extend(cites_edges)
    for ref in refs:
        recursive_edges(base_doi, edges, ref, depth, curr_depth-1)
    if curr_depth < depth:
        for cite in cites:
            if cite != base_doi:
                recursive_edges(base_doi, edges, cite, depth, curr_depth-1)

In [3]:
def build_graph(g: nx.DiGraph, df: pd.DataFrame, depth: int = 5):
    """
    g - base graph.
    df - DataFrame, containing dois of publications
    Algo: With a fixed depth iterate over all publications within a cluster, gathering citations and references.
        From that, get a list of these connections in the form of edges. Then deduplicate and add them to the graph.
    """
    for ind, row in tqdm(df.iterrows()):
        edges = []
        recursive_edges(row["doi"], edges, depth=depth, curr_depth=depth)
        edges = deduplicate_edges(edges)
        g.add_edges_from(edges)
    

In [40]:
pubs = pd.read_csv("../data/clean_pubs.csv")

Перед тем как пытаться собрать данные в одну связную компоненту путем рекурсивного поиска цитирований, я собираюсь кластеризовать наши данные, чтобы немного ускорить процесс поиска

In [41]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

# Download NLTK resources (if not already downloaded)
nltk.download('punkt_tab')
nltk.download('stopwords')

# Initialize stemmer and stop words
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Tokenize and remove stop words
    tokens = nltk.word_tokenize(text.lower())
    tokens = [stemmer.stem(word) for word in tokens if word.isalnum() and word not in stop_words]
    return ' '.join(tokens)

# Apply preprocessing to the abstracts
pubs['processed_abstract'] = pubs['abstract'].apply(preprocess_text)


[nltk_data] Downloading package punkt_tab to /home/vunz/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /home/vunz/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [42]:
# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the processed abstracts
tfidf_matrix = vectorizer.fit_transform(pubs['processed_abstract'])

In [43]:
from sklearn.cluster import KMeans

# Set the number of clusters (you may want to experiment with this value)
num_clusters = 5
kmeans = KMeans(n_clusters=num_clusters, random_state=42)

# Fit K-Means on the TF-IDF matrix
kmeans.fit(tfidf_matrix)

# Assign cluster labels to each publication in the DataFrame
pubs['cluster'] = kmeans.labels_

In [44]:
pubs.groupby(by="cluster").count()[["doi"]].rename({"doi":"count"},axis=1)

,count
cluster,
0,1548
1,5023
2,14465
3,24885
4,18425


In [45]:
pubs

,Unnamed: 0,title,doi,year,abstract,keywords,refs,authors,processed_abstract,cluster
0,0,Studies of Zγ production in association with a...,10.1007/JHEP07(2017)107,2017,The production of a Z boson and a photon in as...,"(""'Electroweak interaction'"", "" 'Hadron-Hadron...","('Eboli O.J.P., Gonzalez-Garcia M.C., Lietti S...","(""'Ahmadov F.'"", "" 'Aleksandrov I.N.'"", "" 'Bed...",product z boson photon associ dijet system stu...,1
1,1,Towards the detection of light and heavy relic...,10.1016/j.ppnp.2011.01.050,2011,The standard Big Bang cosmology predicts that ...,"(""'Neutrino capture'"", "" 'Relic neutrinos'"", ""...","('Giunti C., Kim C.W., Fundamentals of Neutrin...","(""'Šimkovic F.'"",)",standard big bang cosmolog predict univers abu...,3
2,3,Measurement of the underlying event in jet eve...,10.1140/epjc/s10052-014-2965-5,2014,Distributions sensitive to the underlying even...,"('',)",('The underlying event in hard interactions at...,"(""'Ahmadov F.'"", "" 'Aleksandrov I.N.'"", "" 'Bed...",distribut sensit underli event qcd jet event m...,2
3,4,Bubble and kink solitons in the φ6-model of no...,10.1016/0375-9601(93)91074-F,1993,We have studied the φ6-model in the parameter ...,"('',)","('Kosevich, Et al., Sov. J. Low Temp. Phys., 2...","(""'Agüero Granados M.A.'"",)",studi paramet domain 1 relev paramet model cas...,4
4,5,Standard complex for quantum lie algebras,10.1134/1.1432906,2001,"For a quantum Lie algebra Γ, let Γ∧ be its ext...","('',)","('Woronowicz S.L., Publ. RIMS Kyoto, 23, (1987...","(""'Burdik C.'"", "" 'Isaev A.P.'"", "" 'Ogievetsky...",quantum lie algebra γ let exterior extens alge...,4
...,...,...,...,...,...,...,...,...,...,...
64341,72011,Molecular markers development for studying of ...,10.1063/5.0063872,2021,['Here we report about molecular genetic marke...,"('',)","('Philippe H., Et al., Phylogenomics Revives T...","(""'Yakhnenko A.'"", "" 'Yushin N.'"", "" 'Zinicovs...",report molecular genet marker develop primer p...,3
64342,72012,Measurement of the anomalous precession freque...,10.1103/PhysRevD.103.072002,2021,['The Muon g-2 Experiment at Fermi National Ac...,"('',)","('Abi B., Letter companion, Measurement of the...","(""'Baranov V.A.'"", "" 'Duginov V.N.'"", "" 'Khomu...",muon experi fermi nation acceler laboratori fn...,2
64343,72013,ATLAS results on quarkonia and heavy flavor pr...,10.1142/S0217751X20440030,2020,['The associated production a vector boson wit...,"(""'associated production'"", "" 'atlas'"", "" 'hea...","('J. Instrum, 3, (2008)', 'J. High Energy Phys...","(""'Lyubushkina T.'"",)",associ product vector boson key observ underst...,2
64344,72014,Analytical study of light bullets stabilizatio...,10.1016/j.chaos.2022.111799,2022,"[""Dynamics of light bullets in the Raman activ...","(""'diffraction'"", "" 'light bullet'"", "" 'lyapun...","('Couairon A., Mysyrowicz A., Femtosecond fila...","(""'Bugay A.N.'"", "" 'Khalyapin V.A.'"")",dynam light bullet raman activ ioniz medium an...,4


Выберем, к примеру, второй кластер. Потом надо будет использовать алгоритм на каждом, но в данный момент для проверки я просто возьму кластер со сравнительно небольшим количеством публикаций. И не слишком маленьким, как в первом кластере

In [46]:
cluster0 = pubs.loc[pubs.cluster == 0]
samp = cluster0.iloc[:450]

In [13]:
graph = nx.DiGraph()

In [ ]:
# build_graph(graph, samp, depth=2)

19it [06:13, 16.67s/it]

HTTP error: 400 Client Error: Bad Request for url: https://opencitations.net/index/coci/api/v1/citations/10.1016/0029-554X(67)91363-8


21it [06:24, 10.57s/it]

Request failed: HTTPSConnectionPool(host='opencitations.net', port=443): Read timed out. (read timeout=10)
Request failed: HTTPSConnectionPool(host='opencitations.net', port=443): Read timed out. (read timeout=10)
Request failed: HTTPSConnectionPool(host='opencitations.net', port=443): Read timed out. (read timeout=10)
Request failed: HTTPSConnectionPool(host='opencitations.net', port=443): Read timed out. (read timeout=10)


58it [12:38, 11.96s/it]

...

439it [3:20:46,  9.63s/it]

Request failed: HTTPSConnectionPool(host='opencitations.net', port=443): Read timed out. (read timeout=10)


441it [3:21:31, 14.83s/it]

In crossref got an exception timed out


450it [3:24:22, 27.25s/it]


In [6]:
def quick_info(g):
    print(f"Number of resulting nodes: {len(g.nodes)}")
    print(f"Number of edges in resulting graph: {len(g.edges)}")
    if nx.is_strongly_connected(g):
        print("The graph is strongly connected!")
    elif nx.is_weakly_connected(g):
        print("The graph is weakly connected.")
    else:
        print(f"The graph is not connected, having {nx.number_strongly_connected_components(g)} strongly connected components")
        print(f"And {nx.number_weakly_connected_components(g)} weakly connected components")

In [7]:
quick_info(graph)

Number of resulting nodes: 309823
Number of edges in resulting graph: 503819
The graph is not connected, having 309789 strongly connected components
And 39 weakly connected components


In [ ]:
# save graph to graph.pickle
pickle.dump(graph, open("../data/graph.pickle", "wb"))

In [4]:
graph = pickle.load(open("../data/graph.pickle", "rb"))

In [9]:
largest_cc = max(nx.weakly_connected_components(graph))
subgraph = nx.subgraph(graph, largest_cc)

In [10]:
quick_info(subgraph)

Number of resulting nodes: 307888
Number of edges in resulting graph: 501433
The graph is weakly connected.


Теперь посчитаем метрики центральностей

In [11]:
def calc_centralities(g):
    deg = nx.degree_centrality(g)
    bet = nx.betweenness_centrality(g)
    eig = nx.eigenvector_centrality(g)
    clos = nx.closeness_centrality(g)
    pagerank = nx.pagerank(g)
    return {"Degree": deg, "Betweenness": bet, "Eigen": eig, "Closeness": clos, "PageRank": pagerank}

In [ ]:
results = calc_centralities(subgraph)

Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7df8b2dc8f50>>
Traceback (most recent call last):
  File "/home/vunz/Projects/Diploma/jup/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


Для посчета метрик я попросил запустить вычисления на сервере. Поэтому дальше обрабатываю json результаты

In [16]:
import json
import collections

In [6]:
degree = json.load(open("../data/degree.json"))
eigen = json.load(open("../data/eigen.json"))
closeness = json.load(open("../data/closeness.json"))
betweenness = json.load(open("../data/betweenness.json"))
pagerank = json.load(open("../data/pagerank.json"))

In [17]:
degree = collections.OrderedDict(sorted(degree.items()))
eigen = collections.OrderedDict(sorted(eigen.items()))
closeness = collections.OrderedDict(sorted(closeness.items()))
betweenness = collections.OrderedDict(sorted(betweenness.items()))
pagerank = collections.OrderedDict(sorted(pagerank.items()))

In [32]:
df_metrics = pd.DataFrame(degree.values(), index=degree.keys()).rename({0: "degree"}, axis=1)

In [33]:
df_metrics["eigen"] = eigen.values()
df_metrics["closeness"] = closeness.values()
df_metrics["betweenness"] = betweenness.values()
df_metrics["pagerank"] = pagerank.values()

In [35]:
df_metrics.nlargest(10, "degree")

,degree,eigen,closeness,betweenness,pagerank
10.1017/CBO9780511524646,0.018530,5.301566e-22,0.016578,0.000000e+00,0.005365
10.1103/PhysRev.93.99,0.018427,8.474252e-04,0.026207,3.522018e-06,0.005208
10.1103/RevModPhys.57.287,0.018175,1.696595e-14,0.014253,5.865808e-05,0.004879
10.1103/PhysRev.131.2766,0.018030,1.267654e-03,0.027930,7.359271e-06,0.004439
10.1103/PhysRev.115.485,0.017426,1.172719e-02,0.046483,5.679642e-06,0.004692
10.1016/0550-3213(79)90022-1,0.013350,2.890676e-03,0.026376,1.263169e-04,0.002438
10.1006/adnd.1995.1002,0.011003,4.709557e-16,0.011959,0.000000e+00,0.002037
10.1143/JJAP.27.L209,0.010771,8.102113e-26,0.008158,2.996363e-06,0.003158
10.1103/PhysRev.130.2529,0.010367,1.902232e-03,0.028841,4.395669e-06,0.002767
10.1016/0031-9163(62)91369-0,0.010267,2.542276e-03,0.025970,8.714625e-07,0.002733


In [36]:
df_metrics.nlargest(10, "eigen")

,degree,eigen,closeness,betweenness,pagerank
10.1103/PhysRev.83.688,0.000026,0.216720,0.042345,0.000000,0.000035
10.1103/PhysRev.93.233,0.000023,0.154497,0.043223,0.000000,0.000159
10.1103/PhysRev.87.693,0.000029,0.152102,0.042732,0.000000,0.000033
10.1103/PhysRev.87.1100,0.000029,0.151775,0.037266,0.000000,0.000029
10.1103/PhysRev.88.1142,0.000026,0.147518,0.039551,0.000000,0.000028
10.1103/PhysRev.90.497,0.000265,0.139797,0.039791,0.000003,0.000071
10.1103/PhysRev.104.254,0.005920,0.136172,0.051663,0.000016,0.001699
10.1103/RevModPhys.13.203,0.001504,0.134626,0.070009,0.000031,0.000611
10.1103/PhysRev.95.1669,0.000403,0.127787,0.042116,0.000016,0.000102
10.1103/PhysRev.75.1736,0.000013,0.114590,0.036354,0.000000,0.000045


In [37]:
df_metrics.nlargest(10, "closeness")

,degree,eigen,closeness,betweenness,pagerank
10.1103/RevModPhys.13.203,0.001504,0.134626,0.070009,0.000031,0.000611
10.1103/PhysRev.76.790,0.000023,0.109866,0.064020,0.000000,0.000443
10.1103/PhysRev.109.193,0.006223,0.049664,0.063183,0.000074,0.001911
10.1103/RevModPhys.8.82,0.000023,0.038946,0.062279,0.000000,0.000595
10.1016/S0031-8914(40)90099-4,0.000016,0.072516,0.061988,0.000000,0.000408
10.1103/PhysRev.75.486,0.000016,0.049510,0.061230,0.000000,0.000305
10.1007/BF01339504,0.003721,0.069550,0.060623,0.000000,0.001081
10.1016/S0031-8914(39)90090-X,0.001023,0.067188,0.060182,0.000009,0.000424
10.1038/143201a0,0.000006,0.099708,0.059525,0.000000,0.000071
10.1016/S0031-8914(39)90089-3,0.000010,0.067187,0.059116,0.000000,0.000090


In [38]:
df_metrics.nlargest(10, "betweenness")

,degree,eigen,closeness,betweenness,pagerank
10.1016/0370-2693(71)90665-4,0.000817,5.517147e-04,0.032316,0.000290,0.000150
10.1002/prop.19610091102,0.001194,7.558833e-04,0.025299,0.000286,0.000053
10.1016/S0370-1573(87)80002-9,0.007572,1.473134e-08,0.017955,0.000272,0.001692
10.1016/0550-3213(69)90216-8,0.000010,3.029627e-04,0.027460,0.000237,0.000006
10.1016/0550-3213(79)90023-3,0.006881,2.890644e-03,0.024697,0.000223,0.001117
10.1103/RevModPhys.44.284,0.001052,5.418362e-04,0.016370,0.000221,0.000137
10.1007/BF02728133,0.002524,2.167925e-03,0.022490,0.000208,0.000668
10.1016/0370-2693(72)90420-0,0.000804,6.623604e-04,0.033980,0.000205,0.000320
10.1016/0550-3213(72)90190-3,0.000010,1.083870e-03,0.018439,0.000200,0.000118
10.1016/0370-1573(81)90059-4,0.004864,4.025980e-09,0.011487,0.000184,0.000915


In [39]:
df_metrics.nlargest(10, "pagerank")

,degree,eigen,closeness,betweenness,pagerank
10.1017/CBO9780511524646,0.018530,5.301566e-22,0.016578,0.000000e+00,0.005365
10.1103/PhysRev.93.99,0.018427,8.474252e-04,0.026207,3.522018e-06,0.005208
10.1103/RevModPhys.57.287,0.018175,1.696595e-14,0.014253,5.865808e-05,0.004879
10.1103/PhysRev.115.485,0.017426,1.172719e-02,0.046483,5.679642e-06,0.004692
10.1103/PhysRev.131.2766,0.018030,1.267654e-03,0.027930,7.359271e-06,0.004439
10.1016/0370-2693(75)90162-8,0.000026,1.681489e-02,0.033549,0.000000e+00,0.003271
10.1143/JJAP.27.L209,0.010771,8.102113e-26,0.008158,2.996363e-06,0.003158
10.1016/0370-2693(75)90163-X,0.007504,2.132962e-02,0.041031,1.593069e-07,0.002855
10.1088/0022-3719/4/14/022,0.010048,2.498750e-25,0.008344,4.143266e-06,0.002795
10.1103/PhysRev.130.2529,0.010367,1.902232e-03,0.028841,4.395669e-06,0.002767


In [50]:
jinr_subdf = df_metrics[df_metrics.index.isin(samp['doi'])]

In [51]:
jinr_subdf.nlargest(10, "pagerank")

,degree,eigen,closeness,betweenness,pagerank
10.1016/0031-9163(62)90221-4,0.000100,1.497168e-07,0.012742,9.965129e-06,0.000224
10.1016/0031-9163(65)90885-1,0.000939,6.155996e-33,0.000913,9.747595e-07,0.000215
10.1007/BF02777988,0.000261,1.083894e-03,0.018499,5.936431e-06,0.000167
10.1007/BF02742679,0.000381,5.372915e-12,0.003531,4.970233e-07,0.000129
10.1080/00150198008009006,0.000394,2.567050e-33,0.000381,0.000000e+00,0.000103
10.1016/0370-2693(68)90437-1,0.000355,4.310600e-08,0.006791,1.269500e-06,0.000071
10.1007/BF02746567,0.000336,5.506903e-09,0.008671,7.710729e-06,0.000060
10.1007/BF02891914,0.000297,1.119957e-09,0.010095,1.215018e-06,0.000049
10.1103/PhysRevLett.93.142001,0.000239,5.886459e-26,0.000208,2.851118e-08,0.000048
10.1070/RM2001v056n01ABEH000374,0.000242,1.501242e-33,0.000223,0.000000e+00,0.000047


На этом этапе стоит разобраться в корректности аггрегирования метрик для получения одной более интуитивной и уникальной оценки вершины

In [6]:
def agg_centralities():
    pass

Время визуализации. Постараемся нарисовать как можно более понятный рисунок, используя кластеризация и без неё